In [ ]:
# !pip install duckdb

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
import seaborn.objects as sbno
import matplotlib as mpl
from duckdb import query as dbquery
from textwrap import wrap

In [ ]:
GHG = pd.read_csv('Quarterly_Greenhouse_Gas_(GHG)_Air_Emissions_Accounts.csv')
FC = pd.read_csv('Forest_and_Carbon.csv')

In [ ]:
GHG = GHG.drop(columns=["ISO2", "ISO3", "ObjectId", "Source", "CTS_Code", "Scale", \
                        "Indicator", "Unit", "CTS_Name", "CTS_Full_Descriptor"], axis=1)
GHG = GHG.query("Seasonal_Adjustment == 'Seasonally Adjusted'").drop(columns=["Seasonal_Adjustment"], axis=1)

In [ ]:
GHG['Country'].unique()

array(['Advanced Economies', 'Africa', 'Americas', 'Asia',
       'Australia and New Zealand', 'Central Asia', 'Eastern Asia',
       'Eastern Europe', 'Emerging and Developing Economies', 'Europe',
       'G20', 'G7', 'Latin America and the Caribbean', 'Northern Africa',
       'Northern America', 'Northern Europe', 'Oceania',
       'Other Oceania sub-regions', 'South-eastern Asia', 'Southern Asia',
       'Southern Europe', 'Sub-Saharan Africa', 'Western Asia',
       'Western Europe', 'World'], dtype=object)

In [ ]:
countries = GHG['Country'].unique()
industries = GHG['Industry'].unique()
gases = GHG['Gas_Type'].unique()

In [1]:
tranposeDataFrame = lambda df, col_name, start_idx: pd.DataFrame(df.values[:, start_idx:].astype(np.float64).T, columns=df[col_name].to_list())

In [ ]:
def CI_normareaplot(df, country, columns, labels, ax=None):
  df_ = df.query("Country == @country")
  df2 = tranposeDataFrame(df_, columns, 2)
  t = ax is None
  if ax is None:
    fig, ax = plt.subplots(dpi=100)
  n = list(labels)
  base = np.zeros(len(df2))
  x = np.arange(len(df2))
  total = np.zeros(len(df2))
  for col in n:
    if col in df2:
      total += df2[col].values

  for col in n:
    y = np.zeros_like(base)
    if col in df2:
      y = df2[col].values
    ax.fill_between(x, base/total, (base+y)/total, label='\n'.join(wrap(col, 30)))
    base += y

  ax.set_xlabel("Quarters since 2010")
  ax.set_ylabel("Percentage")
  ax.set_title(country)
  ax.set_xlim(0, len(df2)-1)
  ax.set_ylim(0, 1)

  if t:
    ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
def CI_areaplot(df, country, columns, labels, ax=None):
  df_ = df.query("Country == @country")
  df2 = tranposeDataFrame(df_, columns, 2)
  t = ax is None

  if ax is None:
    fig, ax = plt.subplots(dpi=100)
  n = list(labels)
  base = np.zeros(len(df2))
  x = np.arange(len(df2))
  for col in n:
    y = np.zeros_like(base)
    if col in df2:
      y = df2[col].values
    ax.fill_between(x, base, base+y, label='\n'.join(wrap(col, 30)))
    base += y

  ax.set_xlabel("Quarters since 2010")
  ax.set_ylabel("Million Tonnes CO2 equivalent")
  ax.set_title(country)
  ax.set_xlim(0, len(df2)-1)
  ax.set_ylim(0, np.max(base)*1.05)

  if t:
    ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left", fontsize='small')

In [ ]:
GHG_CG = GHG.groupby(['Country', "Gas_Type"], as_index=False).agg('sum').drop(columns=["Industry"], axis=1)

In [ ]:
d2 = GHG_CG.query('Gas_Type != "Greenhouse gas" and Country != "World"')

In [ ]:
fig, axes = plt.subplots(12, 2, figsize=(15, 40), dpi=100)
for country, ax in zip(countries, axes.flat):
  CI_normareaplot(d2, country, "Gas_Type", d2['Gas_Type'].unique(), ax)
hdls, lbls = axes.flat[0].get_legend_handles_labels()
fig.subplots_adjust(wspace=0.2, hspace=0.6)
fig.legend(hdls, lbls, ncols=4, loc='lower center', bbox_to_anchor=(0.5, 0.89))
fig.suptitle("Amount of Greenhouse gases by each region from 2010 to 2023", x=0.5, y=0.91)

In [ ]:
plt.cla()
plt.clf()
plt.close('all')

In [ ]:
GHG_CG = GHG.groupby(['Country', "Industry"], as_index=False).agg('sum').drop(columns=["Gas_Type"], axis=1)

In [ ]:
d2 = dbquery('''
select * from GHG_CG
where Industry not in ('Total Households', 'Total Industry and Households') and Country != 'World';
''').to_df()

In [ ]:
plt.cla()
plt.clf()
plt.close('all')

In [ ]:
fig, axes = plt.subplots(12, 2, figsize=(15, 41), dpi=100)
for country, ax in zip(countries, axes.flat):
  CI_areaplot(d2, country, "Industry", d2['Industry'].unique(), ax)
hdls, lbls = axes.flat[0].get_legend_handles_labels()
fig.subplots_adjust(wspace=0.2, hspace=0.6)
fig.legend(hdls, lbls, ncols=2, loc='lower center', bbox_to_anchor=(0.5, 0.89))
fig.suptitle("Amount of Greenhouse gases by each region from 2010 to 2023", x=0.5, y=0.93)

In [ ]:
GHG_W = GHG.query("Country == 'World'")

In [ ]:
GHG_W_CI = GHG_W.groupby(['Country', "Industry"], as_index=False).agg('sum').drop(columns=["Gas_Type"], axis=1)
d = dbquery('''
select * from GHG_W_CI where Industry not in ('Total Households', 'Total Industry and Households');
''').to_df()

fig, ax = plt.subplots(2, figsize=(8, 10), dpi=120)
CI_areaplot(d, "World", "Industry", d['Industry'].unique(), ax=ax[0])
ax[0].legend(bbox_to_anchor=(1, 1), loc="upper left", fontsize='small')
CI_normareaplot(d, "World", "Industry", d['Industry'].unique(), ax=ax[1])
ax[0].set_title("Greenhouse gas worldwide by industries")
ax[1].set_title("Greenhouse gas worldwide by industries (100-percent)")
fig.subplots_adjust(hspace=0.3)

In [ ]:
GHG_W_CI = GHG_W.groupby(['Country', "Gas_Type"], as_index=False).agg('sum').drop(columns=["Industry"], axis=1)
d = dbquery('''
select * from GHG_W_CI where Gas_Type != 'Greenhouse gas';
''').to_df()

fig, ax = plt.subplots(2, figsize=(8, 10), dpi=120)
CI_areaplot(d, "World", "Gas_Type", d['Gas_Type'].unique(), ax=ax[0])
ax[0].legend(bbox_to_anchor=(1, 1), loc="upper left", fontsize='small')
CI_normareaplot(d, "World", "Gas_Type", d['Gas_Type'].unique(), ax=ax[1])
ax[0].set_title("Greenhouse gas worldwide all industries")
ax[1].set_title("Greenhouse gas worldwide all industries (100-percent)")
fig.subplots_adjust(hspace=0.3)

In [ ]:
FC = FC.drop(columns=['ISO2', 'ISO3', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'Source', 'ObjectId']) \
       .query('Indicator not in ["Index of carbon stocks in forests", "Index of forest extent"]')

In [ ]:
FC.info()

In [ ]:
FC2 = FC.ffill(axis=1)

In [ ]:
dbquery('''
select distinct Country from FC
where F2011 is null
''')

┌───────────────────────┐
│        Country        │
│        varchar        │
├───────────────────────┤
│ Netherlands Antilles  │
│ Serbia and Montenegro │
└───────────────────────┘

In [ ]:
dbquery('''
select Country, F2020 from FC2
where Country in ('Netherlands Antilles', 'Serbia and Montenegro')
''')

┌───────────────────────┬──────────────────┐
│        Country        │      F2020       │
│        varchar        │      double      │
├───────────────────────┼──────────────────┤
│ Netherlands Antilles  │           0.0032 │
│ Netherlands Antilles  │             2.35 │
│ Netherlands Antilles  │             80.0 │
│ Netherlands Antilles  │           2.9375 │
│ Serbia and Montenegro │         236.7389 │
│ Serbia and Montenegro │           3313.0 │
│ Serbia and Montenegro │          10200.0 │
│ Serbia and Montenegro │ 32.4803921568627 │
└───────────────────────┴──────────────────┘

In [ ]:
real_countries = dbquery('''
select distinct Country from FC2 where
Country not in (
  select distinct Country from GHG
)
''').to_df()

regions = dbquery('''
select distinct Country from FC2 where
Country in (
  select distinct Country from GHG
)
''').to_df()

In [ ]:
regions

,Country
0,Oceania
1,Advanced Economies
2,Asia
3,G20
4,World
5,Americas
6,G7
7,Emerging and Developing Economies
8,Africa


In [ ]:
unit_indicator = dbquery('''
select Indicator, Unit from FC2
where Country = 'World';
''').to_df()

In [ ]:
single_country_forest = tranposeDataFrame(dbquery('''
select * from FC2 where Country = 'World';
''').to_df(), "Indicator", 3)

# single_country_forest = tranposeDataFrame(dbquery('''
# select * from FC2 where Country = 'Vietnam';
# ''').to_df(), "Indicator", 3)

In [ ]:
columns = [
  'Forest area', 'Land area', 'Carbon stocks in forests', 'Share of forest area'
]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8), dpi=120)
for col, ax in zip(columns, axes.flat):
  data = single_country_forest[col]
  ax.set_title(col)
  ax.set_xlabel("Year")
  ax.set_xlim(1992, 2020)
  ylabl = unit_indicator.query('Indicator == @col')['Unit'].iloc[0]
  data2 = None
  if 'HA' in ylabl:
    data2 = data / 1e6
    ylabl = 'billions HA'
  ax.set_ylabel(ylabl)
  ax.plot(np.arange(1992, 2021), data if data2 is None else data2)

fig.subplots_adjust(wspace=0.25, hspace=0.33)
fig.suptitle("Worldwide trends of forest indicators", y=0.95)

In [ ]:
regions_CS = tranposeDataFrame(dbquery('''
select * from FC2
where Country in (select Country from regions) and Indicator = 'Forest area';
''').to_df(), "Country", 3)

fig, axes = plt.subplots(4, 2, figsize=(12, 16), dpi=120)
for col, ax in zip(regions_CS.columns, axes.flat):
  ax.plot(np.arange(1992, 2021), regions_CS[col])
  ax.set_title(col)
  ax.set_xlabel("Year")
  ax.set_xlim(1992, 2020)
  ax.set_ylabel("Percent")

fig.subplots_adjust(hspace=0.31, wspace=0.23)
fig.suptitle("Forest area by regions", y=0.91)